In [23]:
import os
import subprocess
import pandas as pd 
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')
outdir = 'results/refs/ensembl/'
os.makedirs(outdir, exist_ok=True)

fn = os.path.join(outdir, 'gencode.v19.annotation.gtf.gz')
if not os.path.exists(fn):
    url = 'https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_19/gencode.v19.annotation.gtf.gz'
    cmd = 'wget -O {} {}'.format(fn, url)
    output = subprocess.check_output(cmd, shell=True)

In [24]:
gtf = pd.read_table(fn, header=None, skiprows=5)

In [25]:
gtf.head()

,0,1,2,3,4,5,6,7,8
0,chr1,HAVANA,gene,11869,14412,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
1,chr1,HAVANA,transcript,11869,14409,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
2,chr1,HAVANA,exon,11869,12227,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
3,chr1,HAVANA,exon,12613,12721,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
4,chr1,HAVANA,exon,13221,14409,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."


In [49]:
# extract the main columns
parsed = gtf.iloc[:, [0,3,4,8,6,2]]

In [50]:
# get the gene id and gene name
parsed['gene_id'] = parsed[8].str.extract('gene_id "(ENSG[0-9.]+)"')
parsed['gene_name'] = parsed[8].str.extract('gene_name "([A-Za-z0-9-\.]+)"')

# final data cleanup
parsed = parsed.iloc[:, [0,1,2,4,5,6,7]]

/mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
# use the gene ID as the name when the name is not 
# available
new_genenames = []
for ID, name in parsed.values[:, 5:7]:
    if type(name) == str:
        new_genenames.append(name)
    else:
        new_genenames.append(ID)
parsed.loc[:, 'gene_name'] = new_genenames

In [54]:
# save the final output
bed = os.path.join(outdir, 'gencode.v19.annotation.bed')
parsed.to_csv(bed, sep='\t', header=None, index=False)

In [55]:
parsed[parsed.gene_name == 'AP003774.1']

,0,3,4,6,2,gene_id,gene_name
1409358,chr11,64092522,64096976,-,gene,ENSG00000236935.1,AP003774.1
1409359,chr11,64092522,64096976,-,transcript,ENSG00000236935.1,AP003774.1
1409360,chr11,64096640,64096976,-,exon,ENSG00000236935.1,AP003774.1
1409361,chr11,64094956,64095008,-,exon,ENSG00000236935.1,AP003774.1
1409362,chr11,64092522,64092803,-,exon,ENSG00000236935.1,AP003774.1
